In [1]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/UTCA_US_AF_superFILTRO_sample_inputmatrix.txt',sep='\t')

In [2]:
BRCA_FR_SUPERFILTRATO_WG

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,UTCA-FR,SA598014,SA598014,GRCh37,SNP,2,57996153,57996153,G,C,SOMATIC
1,UTCA-FR,SA598014,SA598014,GRCh37,SNP,2,57996153,57996153,G,C,SOMATIC
2,UTCA-FR,SA598012,SA598012,GRCh37,SNP,3,6930587,6930587,A,T,SOMATIC
3,UTCA-FR,SA598012,SA598012,GRCh37,SNP,14,40855237,40855237,A,T,SOMATIC
4,UTCA-FR,SA598012,SA598012,GRCh37,SNP,14,40855237,40855237,A,T,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
283415,UTCA-FR,SA598002,SA598002,GRCh37,SNP,9,99166175,99166175,C,T,SOMATIC
283416,UTCA-FR,SA598003,SA598003,GRCh37,SNP,8,63400299,63400299,C,T,SOMATIC
283417,UTCA-FR,SA598002,SA598002,GRCh37,SNP,8,63400299,63400299,C,T,SOMATIC
283418,UTCA-FR,SA598002,SA598002,GRCh37,SNP,20,23691266,23691266,G,C,SOMATIC


In [3]:
ESONI=pd.read_csv('/content/drive/MyDrive/ESONI.tsv',sep='\t')

In [4]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [5]:
ESONI['chrom']=ESONI['chrom'].astype(str)

In [6]:
ESONI['exonStarts']=ESONI['exonStarts'].astype(int)

In [7]:
ESONI['exonEnds']=ESONI['exonEnds'].astype(int)

In [8]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [9]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Definizione della funzione per il confronto dei blocchi
def compare_blocks(block_BRCA, block_ESONI):
    # Resto del codice rimane invariato
    merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
    filtered_block = merged_block[
        (merged_block['pos_start'] >= merged_block['exonStarts']) &
        (merged_block['pos_end'] <= merged_block['exonEnds'])
    ]
    return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start',
       'pos_end', 'ref', 'alt', 'Type']]

# Dimensione dei blocchi
block_size = 1000

# Calcolo del numero di blocchi per ciascun DataFrame
num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

# Creazione della lista di indici per i blocchi
block_indices = []
for i in range(num_blocks_BRCA):
    start_idx_BRCA = i * block_size
    end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

    for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

# Funzione wrapper per la chiamata a compare_blocks con gli indici
def compare_blocks_wrapper(indices):
    block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
    block_ESONI = ESONI.iloc[indices[2]:indices[3]]

    return compare_blocks(block_BRCA, block_ESONI)

# Utilizzo di multiprocessing per eseguire operazioni parallele
with Pool() as pool:
    results = pool.map(compare_blocks_wrapper, block_indices)

# Combinazione dei risultati in un unico DataFrame
final_result = pd.concat(results, ignore_index=True)

In [10]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,UTCA-FR,SA598014,SA598014,GRCh37,SNP,chr1,220986917,220986917,C,A,SOMATIC
1,UTCA-FR,SA598014,SA598014,GRCh37,SNP,chr1,220986917,220986917,C,A,SOMATIC
2,UTCA-FR,SA598012,SA598012,GRCh37,SNP,chr1,220986917,220986917,C,A,SOMATIC
3,UTCA-FR,SA598012,SA598012,GRCh37,SNP,chr1,220986917,220986917,C,A,SOMATIC
4,UTCA-FR,SA598012,SA598012,GRCh37,SNP,chr1,233105781,233105781,C,T,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
66147,UTCA-FR,SA598003,SA598003,GRCh37,SNP,chr20,62328732,62328732,C,T,SOMATIC
66148,UTCA-FR,SA598003,SA598003,GRCh37,SNP,chr20,62328732,62328732,C,T,SOMATIC
66149,UTCA-FR,SA598003,SA598003,GRCh37,SNP,chr20,62328732,62328732,C,T,SOMATIC
66150,UTCA-FR,SA598003,SA598003,GRCh37,SNP,chr20,62328732,62328732,C,T,SOMATIC


In [11]:
for i in range(final_result.shape[0]):
  final_result.loc[i,'chrom']=final_result.loc[i,'chrom'][3:]

In [12]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,UTCA-FR,SA598014,SA598014,GRCh37,SNP,1,220986917,220986917,C,A,SOMATIC
1,UTCA-FR,SA598014,SA598014,GRCh37,SNP,1,220986917,220986917,C,A,SOMATIC
2,UTCA-FR,SA598012,SA598012,GRCh37,SNP,1,220986917,220986917,C,A,SOMATIC
3,UTCA-FR,SA598012,SA598012,GRCh37,SNP,1,220986917,220986917,C,A,SOMATIC
4,UTCA-FR,SA598012,SA598012,GRCh37,SNP,1,233105781,233105781,C,T,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
66147,UTCA-FR,SA598003,SA598003,GRCh37,SNP,20,62328732,62328732,C,T,SOMATIC
66148,UTCA-FR,SA598003,SA598003,GRCh37,SNP,20,62328732,62328732,C,T,SOMATIC
66149,UTCA-FR,SA598003,SA598003,GRCh37,SNP,20,62328732,62328732,C,T,SOMATIC
66150,UTCA-FR,SA598003,SA598003,GRCh37,SNP,20,62328732,62328732,C,T,SOMATIC


In [13]:
final_result.to_csv('utca_us_we_input.txt', sep='\t', index=False)

In [14]:
from google.colab import files
files.download('utca_us_we_input.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>